In [4]:
import pandas as pd
from tabulate import tabulate
from fetch_process_function import fetch_and_save_erc20_transfers, process_transactions

In [5]:
def calculate_pnl(btc_record_df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate PnL for BTC transactions based on BTC_record_df.

    :param btc_record_df: DataFrame containing BTC transaction records.
    :return: DataFrame with calculated PnL and summary of Realised and Unrealised PnL.
    """
    
    # Define current price as the price of the last row
    current_price = btc_record_df.iloc[-1]['average_price']
    
    results = []
    total_pnl = 0

    for _, row in btc_record_df.iterrows():
        transaction_type = row['transaction_type']
        amount = row['other_token_volume']
        price = row['average_price']

        # Calculate PnL for each row based on the difference from the current price
        pnl = (current_price - price) * amount
        
        # For SELL transactions, PnL is negative because it's reducing the position
        if transaction_type == "SELL":
            pnl *= -1

        # Ignore the last row (current price reference)
        if row.name == btc_record_df.index[-1]:
            pnl = 0

        total_pnl += pnl

        # Append the result for this transaction
        results.append({
            "Date": row['dateTime'],
            "timeStamp": row['timeStamp'],
            "Type": transaction_type,
            "amount": amount,
            "price": price,
            "pnl": pnl
        })

    # Convert results to a DataFrame
    pnl_df = pd.DataFrame(results)

    return pnl_df


In [6]:
ADDRESS = "0x5be9a4959308A0D0c7bC0870E319314d8D957dBB"
START_DATE = "2024-12-06 08:00" # Singapore Time，the function will convert it to UTC
END_DATE = "2024-12-12 08:00"
OUTPUT_FILE = "C:/Users/YuweiCao/Documents/GitHub/Project/Project/etherscan/result"
API_KEY = "VQAIR728IM4Z8RZKPYBR4ESM5I3WBZK2C1"
BASE_TOKENS = {"USDT", "USDC", "USDE"}
# transfers_df = fetch_and_save_erc20_transfers(
#         address=ADDRESS,
#         start_date=START_DATE,
#         end_date=END_DATE,
#         output_file=OUTPUT_FILE,
#         offset=1000,
#     )
transfers_df = pd.read_csv(OUTPUT_FILE + "/all_erc20_transfers.csv")
record_df_1, record_df_2, record_df_3 = process_transactions(transfers_df, OUTPUT_FILE, ADDRESS, BASE_TOKENS)
# print(tabulate(summary, headers='keys', tablefmt='grid', showindex=False, floatfmt=".2f"))
unique_tokens_count = record_df_2["token_symbol"].nunique()

# 查看所有的代币种类
unique_tokens = record_df_2["token_symbol"].unique()

# 打印结果
print(f"Total unique tokens: {unique_tokens_count}")
print(f"List of tokens: {unique_tokens}")

BTC_record_df = record_df_1[record_df_1["formatted_record"].str.contains(r"\bcbBTC\b", regex=True)]
# print(BTC_record_df['formatted_record'])
pnl_df = calculate_pnl(BTC_record_df)
print(f"Total PnL: {pnl_df['pnl'].sum()}")



Formatted Transactions: part 1
Formatted Transactions: part 2
Formatted Transactions: final part
Total unique tokens: 4
List of tokens: ['WLFI' 'cbBTC' 'WETH' 'OPTIMUS']
Total PnL: 843.2596881283655
